In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.9 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification

In [3]:
# Load the dataset
# Assuming the dataset is in a file named 'dataset.xlr'
data = pd.read_excel('/content/Cricket.xlsx')

In [4]:
# Split the dataset into input texts and corresponding labels
texts = data['Text'].values
labels = data['Polarity'].values

In [5]:
# Load the DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

In [6]:
# Tokenize the input texts
encoded_texts = tokenizer(texts.tolist(), padding=True, truncation=True, return_tensors='tf')

In [7]:
# Convert labels to integers (positive: 1, negative: 0)
labels = np.array([1 if label == 'positive' else 0 for label in labels])

In [8]:
# Split the data into training and validation sets
train_size = int(0.8 * len(encoded_texts['input_ids']))
train_inputs = {key: val[:train_size] for key, val in encoded_texts.items()}
train_labels = labels[:train_size]

val_inputs = {key: val[train_size:] for key, val in encoded_texts.items()}
val_labels = labels[train_size:]

In [9]:
# Load the pre-trained DistilBERT model for sequence classification
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=2)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [10]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [11]:
# Convert inputs and labels to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_inputs, train_labels)).shuffle(len(train_labels)).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((val_inputs, val_labels)).batch(32)

In [12]:
# Train the model using the datasets
model.fit(train_dataset, validation_data=val_dataset, epochs=5)

Epoch 1/5
75/75 [==============================] - 1672s 22s/step - loss: 0.5165 - accuracy: 0.7733 - val_loss: 0.3113 - val_accuracy: 0.8993
Epoch 2/5
75/75 [==============================] - 1676s 22s/step - loss: 0.4075 - accuracy: 0.8249 - val_loss: 0.2837 - val_accuracy: 0.8893
Epoch 3/5
75/75 [==============================] - 1682s 22s/step - loss: 0.3493 - accuracy: 0.8619 - val_loss: 0.4030 - val_accuracy: 0.8289
Epoch 4/5
75/75 [==============================] - 1664s 22s/step - loss: 0.2828 - accuracy: 0.8900 - val_loss: 0.4052 - val_accuracy: 0.8322
Epoch 5/5
75/75 [==============================] - 1708s 23s/step - loss: 0.2035 - accuracy: 0.9291 - val_loss: 0.3908 - val_accuracy: 0.8423


In [13]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(val_inputs, val_labels)
print("Validation Accuracy:", accuracy)

19/19 [==============================] - 105s 5s/step - loss: 0.3908 - accuracy: 0.8423
Validation Accuracy: 0.8422818779945374
